In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

load_dotenv()
connection = os.environ.get("POSTGRES_ENGINE")

engine = create_engine(connection)

In [3]:
user_data = pd.read_sql('SELECT * FROM public.user_data', con=engine)
post_text = pd.read_sql('SELECT * FROM public.post_text_df', con=engine)

In [4]:
feed_data = pd.read_sql('SELECT * FROM public.feed_data limit 1000000', con=engine)
feed_data.shape

(1000000, 5)

In [5]:
feed_data.head()

,timestamp,user_id,post_id,action,target
0,2021-11-27 16:44:20,109008,6259,view,0
1,2021-11-27 16:44:55,109008,6936,view,0
2,2021-11-27 16:45:49,109008,6855,view,0
3,2021-11-27 16:47:50,109008,5011,view,0
4,2021-11-27 16:50:30,109008,5945,view,1


In [6]:
feed_data = feed_data.drop(['timestamp', 'target'], axis=1)
feed_data.head()

,user_id,post_id,action
0,109008,6259,view
1,109008,6936,view
2,109008,6855,view
3,109008,5011,view
4,109008,5945,view


In [2]:
distinct_feed_like = pd.read_sql("select distinct on (user_id) user_id, post_id, action from public.feed_data where action = 'like'", con=engine)
distinct_feed_like.head()

,user_id,post_id,action
0,200,4345,like
1,201,154,like
2,202,3799,like
3,203,630,like
4,204,2084,like


In [3]:
distinct_feed_view = pd.read_sql("select distinct on (user_id) user_id, post_id, action from public.feed_data where action = 'view'", con=engine)
distinct_feed_view.head()

,user_id,post_id,action
0,200,1185,view
1,201,1977,view
2,202,2418,view
3,203,4022,view
4,204,3328,view


In [9]:
feed_data = pd.concat([feed_data, distinct_feed_like, distinct_feed_view], ignore_index=True)
feed_data.head()

,user_id,post_id,action
0,109008,6259,view
1,109008,6936,view
2,109008,6855,view
3,109008,5011,view
4,109008,5945,view


In [10]:
feed_data.shape

(1326407, 3)

In [11]:
feed_data.drop_duplicates(inplace=True)
feed_data.shape

(1280551, 3)

In [12]:
user_data.head()

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


In [13]:
post_text.head()

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business


In [14]:
countries = list(user_data.country.unique())
topics = list(post_text.topic.unique())
OSes = list(user_data.os.unique())

In [15]:
post_text['text_length'] = post_text['text'].str.len()
post_text = post_text.drop('text', axis=1)

In [16]:
for country in countries:
    user_data[country] = (
        user_data['country']
        .str
        .contains(country)
        .apply(int)
    )

user_data = user_data.drop('country', axis=1)

for os in OSes:
    user_data[os] = (
        user_data['os']
        .str
        .contains(os)
        .apply(int)
    )

user_data = user_data.drop(['os', 'source', 'city'], axis=1)

user_data.head()

,user_id,gender,age,exp_group,Russia,Ukraine,Belarus,Azerbaijan,Kazakhstan,Finland,Turkey,Latvia,Cyprus,Switzerland,Estonia,Android,iOS
0,200,1,34,3,1,0,0,0,0,0,0,0,0,0,0,1,0
1,201,0,37,0,1,0,0,0,0,0,0,0,0,0,0,1,0
2,202,1,17,4,1,0,0,0,0,0,0,0,0,0,0,1,0
3,203,0,18,1,1,0,0,0,0,0,0,0,0,0,0,0,1
4,204,0,36,3,1,0,0,0,0,0,0,0,0,0,0,1,0


In [17]:
for topic in topics:
    post_text[topic] = (
        post_text['topic']
        .str
        .contains(topic)
        .apply(int)
    )

post_text = post_text.drop('topic', axis=1)

post_text.head()

,post_id,text_length,business,covid,entertainment,sport,politics,tech,movie
0,1,1967,1,0,0,0,0,0,0
1,2,2701,1,0,0,0,0,0,0
2,3,3408,1,0,0,0,0,0,0
3,4,1026,1,0,0,0,0,0,0
4,5,889,1,0,0,0,0,0,0


In [21]:
user_data.to_sql('m_mishin_features_lesson_22_user', con=engine, if_exists='replace', index=False)

205

In [23]:
post_text.to_sql('m_mishin_features_lesson_22_post', con=engine, if_exists='replace', index=False)

23

In [22]:
user_test = pd.read_sql('m_mishin_features_lesson_22_user', con=engine)
user_test

,user_id,gender,age,exp_group,Russia,Ukraine,Belarus,Azerbaijan,Kazakhstan,Finland,Turkey,Latvia,Cyprus,Switzerland,Estonia,Android,iOS
0,200,1,34,3,1,0,0,0,0,0,0,0,0,0,0,1,0
1,201,0,37,0,1,0,0,0,0,0,0,0,0,0,0,1,0
2,202,1,17,4,1,0,0,0,0,0,0,0,0,0,0,1,0
3,203,0,18,1,1,0,0,0,0,0,0,0,0,0,0,0,1
4,204,0,36,3,1,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163200,168548,0,36,4,1,0,0,0,0,0,0,0,0,0,0,1,0
163201,168549,0,18,2,1,0,0,0,0,0,0,0,0,0,0,1,0
163202,168550,1,41,4,1,0,0,0,0,0,0,0,0,0,0,1,0
163203,168551,0,38,3,1,0,0,0,0,0,0,0,0,0,0,0,1


In [24]:
post_test = pd.read_sql('m_mishin_features_lesson_22_post', con=engine)

In [92]:
post_text_test = pd.read_sql('SELECT * FROM public.post_text_df', con=engine)
post_text_test

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business
...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie
7019,7316,I give this movie 2 stars purely because of it...,movie
7020,7317,I cant believe this film was allowed to be mad...,movie
7021,7318,The version I saw of this film was the Blockbu...,movie


In [93]:
post_text_test = post_text_test.set_index('post_id')
# post_text_test.set_index('post_id', inplace=True)
post_text_test

,text,topic
post_id,,
1,UK economy facing major risks\n\nThe UK manufa...,business
2,Aids and climate top Davos agenda\n\nClimate c...,business
3,Asian quake hits European shares\n\nShares in ...,business
4,India power shares jump on debut\n\nShares in ...,business
5,Lacroix label bought by US firm\n\nLuxury good...,business
...,...,...
7315,"OK, I would not normally watch a Farrelly brot...",movie
7316,I give this movie 2 stars purely because of it...,movie
7317,I cant believe this film was allowed to be mad...,movie


In [126]:
result = post_text_test.loc[post_ids]
result = result.reset_index().rename(columns={'post_id': 'id'})
# result = result.rename(columns={'post_id': 'id'})
result.head()

,id,text,topic
0,84,US economy still growing says Fed\n\nMost area...,business
1,2370,The new charnamrit is #sanitizer #covid #covid19,covid
2,2426,#COVID19 care-takers https://t.co/Or2jda63Kx,covid
3,2540,Anybody I know test positive for #COVID19 ? #L...,covid
4,4491,This film is NOT about a cat and mouse fight a...,movie


In [123]:
import json
result_ = result.to_json(orient='records')
parsed = json.loads(result_)

parsed

[{'post_id': 84,
  'text': 'US economy still growing says Fed\n\nMost areas of the US saw their economy continue to expand in December and early January, the US Federal Reserve said in its latest Beige Book report.\n\nOf the 12 US regions it identifies for the study, 11 showed stronger economic growth, with only the Cleveland area falling behind with a mixed rating. Consumer spending was higher in December than November, and festive sales were also up on 2003. The employment picture also improved, the Fed said.\n\nLabour markets firmed in a number of districts, but wage pressures generally remained modest, the Beige Book said. Several districts reported higher prices for building materials and manufacturing inputs, but most reported steady or only slightly higher overall price levels. The report added that residential real estate activity remained strong and that commercial real estate activity strengthened in most districts. Office leasing was especially brisk in Washington DC, and Ne

In [28]:
data = pd.merge(feed_data, post_text, on='post_id', how='left')
data = pd.merge(data, user_data, how='left', on='user_id')
data.head()

,user_id,post_id,action,text_length,business,covid,entertainment,sport,politics,tech,...,Azerbaijan,Kazakhstan,Finland,Turkey,Latvia,Cyprus,Switzerland,Estonia,Android,iOS
0,109008,6259,view,663,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,109008,6936,view,873,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,109008,6855,view,2072,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,109008,5011,view,1181,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,109008,5945,view,962,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [29]:
feed_data.user_id.nunique()

163205

In [30]:
data['target'] = 0
data.loc[data['action'] == 'like', 'target'] = 1
data.head()

,user_id,post_id,action,text_length,business,covid,entertainment,sport,politics,tech,...,Kazakhstan,Finland,Turkey,Latvia,Cyprus,Switzerland,Estonia,Android,iOS,target
0,109008,6259,view,663,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,109008,6936,view,873,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,109008,6855,view,2072,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,109008,5011,view,1181,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,109008,5945,view,962,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [31]:
data = data.drop('action', axis=1)

In [32]:
test = pd.read_sql('SELECT * FROM public.feed_data limit 100000', con=engine)
test = test.drop(['timestamp', 'target'], axis=1)
test = pd.merge(test, post_text, on='post_id', how='left')
test = pd.merge(test, user_data, how='left', on='user_id')
test['target'] = 0
test.loc[test['action'] == 'like', 'target'] = 1
test = test.drop('action', axis=1)

test.head()


,user_id,post_id,text_length,business,covid,entertainment,sport,politics,tech,movie,...,Kazakhstan,Finland,Turkey,Latvia,Cyprus,Switzerland,Estonia,Android,iOS,target
0,109142,1157,1708,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,109142,4889,903,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,109142,1881,3338,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,109142,226,2306,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,109142,1745,1588,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [33]:
train = data

In [36]:
X_train = train.drop('target', axis=1)
X_test = test.drop('target', axis=1)
y_train = train['target']
y_test = test['target']

In [62]:
column_names = X_train.columns
column_names_list = ['user_id', 'post_id', 'text_length', 'business', 'covid',
       'entertainment', 'sport', 'politics', 'tech', 'movie', 'gender', 'age',
       'exp_group', 'Russia', 'Ukraine', 'Belarus', 'Azerbaijan', 'Kazakhstan',
       'Finland', 'Turkey', 'Latvia', 'Cyprus', 'Switzerland', 'Estonia',
       'Android', 'iOS']
column_names

Index(['user_id', 'post_id', 'text_length', 'business', 'covid',
       'entertainment', 'sport', 'politics', 'tech', 'movie', 'gender', 'age',
       'exp_group', 'Russia', 'Ukraine', 'Belarus', 'Azerbaijan', 'Kazakhstan',
       'Finland', 'Turkey', 'Latvia', 'Cyprus', 'Switzerland', 'Estonia',
       'Android', 'iOS'],
      dtype='object')

In [50]:
exmp_user_id = 322
single_user_data = user_test.loc[user_test['user_id']==exmp_user_id]
single_user_data

,user_id,gender,age,exp_group,Russia,Ukraine,Belarus,Azerbaijan,Kazakhstan,Finland,Turkey,Latvia,Cyprus,Switzerland,Estonia,Android,iOS
122,322,0,51,2,1,0,0,0,0,0,0,0,0,0,0,1,0


In [55]:
usrs =  pd.concat([single_user_data]*post_test.shape[0], ignore_index=True)
usrs.shape

(7023, 17)

In [57]:
usrs_post = pd.merge(usrs, post_test, left_index=True, right_index=True)
usrs_post

,user_id,gender,age,exp_group,Russia,Ukraine,Belarus,Azerbaijan,Kazakhstan,Finland,...,iOS,post_id,text_length,business,covid,entertainment,sport,politics,tech,movie
0,322,0,51,2,1,0,0,0,0,0,...,0,1,1967,1,0,0,0,0,0,0
1,322,0,51,2,1,0,0,0,0,0,...,0,2,2701,1,0,0,0,0,0,0
2,322,0,51,2,1,0,0,0,0,0,...,0,3,3408,1,0,0,0,0,0,0
3,322,0,51,2,1,0,0,0,0,0,...,0,4,1026,1,0,0,0,0,0,0
4,322,0,51,2,1,0,0,0,0,0,...,0,5,889,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,322,0,51,2,1,0,0,0,0,0,...,0,7315,803,0,0,0,0,0,0,1
7019,322,0,51,2,1,0,0,0,0,0,...,0,7316,800,0,0,0,0,0,0,1
7020,322,0,51,2,1,0,0,0,0,0,...,0,7317,636,0,0,0,0,0,0,1
7021,322,0,51,2,1,0,0,0,0,0,...,0,7318,728,0,0,0,0,0,0,1


In [64]:
user_features = usrs_post.loc[:, column_names_list]
user_features

,user_id,post_id,text_length,business,covid,entertainment,sport,politics,tech,movie,...,Azerbaijan,Kazakhstan,Finland,Turkey,Latvia,Cyprus,Switzerland,Estonia,Android,iOS
0,322,1,1967,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,322,2,2701,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,322,3,3408,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,322,4,1026,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,322,5,889,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,322,7315,803,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
7019,322,7316,800,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
7020,322,7317,636,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
7021,322,7318,728,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0


In [65]:
from catboost import CatBoostClassifier

catboost = CatBoostClassifier()

# catboost.fit(X_train, y_train, cat_features=['exp_group'])
catboost.load_model('model')

In [73]:
user_features['preds'] = catboost.predict_proba(user_features)[:,1]
user_features

,user_id,post_id,text_length,business,covid,entertainment,sport,politics,tech,movie,...,Kazakhstan,Finland,Turkey,Latvia,Cyprus,Switzerland,Estonia,Android,iOS,preds
0,322,1,1967,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0.665329
1,322,2,2701,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0.760399
2,322,3,3408,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0.670562
3,322,4,1026,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0.689300
4,322,5,889,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0.620227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,322,7315,803,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0.425172
7019,322,7316,800,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0.476179
7020,322,7317,636,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0.434765
7021,322,7318,728,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0.539901


In [79]:
post_ids = list(user_features.sort_values('preds', ascending=False).head()['post_id'])
post_ids

[84, 2370, 2426, 2540, 4491]

In [80]:
post_test.loc[post_ids]

,post_id,text_length,business,covid,entertainment,sport,politics,tech,movie
84,69,2468,1,0,0,0,0,0,0
2370,1943,2974,0,0,0,0,0,1,0
2426,1987,2615,0,0,0,0,0,1,0
2540,2062,16063,0,0,0,0,0,1,0
4491,4679,921,0,0,0,0,0,0,1


In [ ]:
preds = X_test
preds['pred'] = catboost.predict(X_test)
preds['target'] = y_test
preds

In [ ]:
preds[preds['target']==preds['pred']].shape

In [ ]:
catboost.save_model('model')

In [ ]:
model_test = CatBoostClassifier()
model_test.load_model('model')


In [ ]:
model_test.predict(X_test)

In [30]:
X_train.to_sql('m_mishin_features_lesson_22', con=engine, if_exists='replace')

50

In [31]:
df = pd.read_sql('SELECT * FROM m_mishin_features_lesson_22', con=engine) # считываем таблицу
df.head()

,index,user_id,post_id,text_length,business,covid,entertainment,sport,politics,tech,...,Azerbaijan,Kazakhstan,Finland,Turkey,Latvia,Cyprus,Switzerland,Estonia,Android,iOS
0,0,108860,1201,3074,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108860,2677,140,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2,108860,229,1987,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,3,108860,1762,2325,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4,4,108860,6042,918,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
